# Start the Shapash Web App on a sample dataset

<b>With this tutorial you:</b><br />
Understand how shapash works with a simple use case<br />
Start WebApp to understand your model and save these results

Contents:
- Build a Regressor
- Compile Shapash SmartExplainer
- Start Shapash WebApp
- Export synt with to_pandas function
- Save Shapash object in pickle file

Data from Kaggle [House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [17]:
import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

## Building Supervized Model 

In [18]:
from shapash.data.data_loader import data_loading
house_df, house_dict = data_loading('house_prices')

In [19]:
y_df=house_df['SalePrice'].to_frame()
X_df=house_df[house_df.columns.difference(['SalePrice'])]

In [20]:
house_df.shape

(1460, 73)

In [21]:
house_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,2-Story 1946 & Newer,Residential Low Density,8450,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,...,0,0,0,0,0,2,2008,Warranty Deed - Conventional,Normal Sale,208500
2,1-Story 1946 & Newer All Styles,Residential Low Density,9600,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Veenker,...,0,0,0,0,0,5,2007,Warranty Deed - Conventional,Normal Sale,181500
3,2-Story 1946 & Newer,Residential Low Density,11250,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,...,0,0,0,0,0,9,2008,Warranty Deed - Conventional,Normal Sale,223500
4,2-Story 1945 & Older,Residential Low Density,9550,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Corner lot,Gentle slope,Crawford,...,272,0,0,0,0,2,2006,Warranty Deed - Conventional,Abnormal Sale,140000
5,2-Story 1946 & Newer,Residential Low Density,14260,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Northridge,...,0,0,0,0,0,12,2008,Warranty Deed - Conventional,Normal Sale,250000


#### Encoding Categorical Features 

In [22]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

#### Train / Test Split

In [23]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

#### Model Fitting

In [24]:
regressor = LGBMRegressor(n_estimators=100).fit(Xtrain,ytrain)

## Understanding my model with shapash

#### Declare and Compile SmartExplainer 

In [25]:
from shapash import SmartExplainer

In [26]:
xpl = SmartExplainer(
    model=regressor,
    preprocessing=encoder,   # Optional: compile step can use inverse_transform method
    features_dict=house_dict # optional parameter, specifies label for features name 
)

In [27]:
xpl.compile(x=Xtest, y_target=ytest)

In [28]:
xpl.plot.scatter_plot_prediction()

#### Start WebApp

In [29]:
app = xpl.run_app(title_story='House Prices')

/home/78257d/shapash/shapash/webapp/smart_app.py:320: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://slhdg002:8050/


Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/



INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app 'shapash.webapp.smart_app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8050
 * Running on http://172.23.252.69:8050 (Press CTRL+C to quit)
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:54] "GET /assets/material-icons.css?m=1638359269.1955516 HTTP/1.1" 304 -
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:54] "GET /assets/style.css?m=1647943137.2501905 HTTP/1.1" 304 -
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:54] "GET /assets/jquery.js?m=1638359269.1873672 HTTP/1.1" 304 -
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:55] "GET /assets/main.js?m=1638359269.1913173 HTTP/1.1" 304 -
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:55] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:55] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:172.16.95.194 - - [01/Jul/2022 16:30:56] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.

Link to App: <br />
[shapash-monitor link](https://shapash-demo.ossbymaif.fr/)

#### Stop the WebApp after using it

In [16]:
app.kill()

#### Export local explaination in DataFrame

In [15]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
    threshold=5000,
)

In [16]:
summary_df.head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
259,209141.256921,Ground living area square feet,1792,13710.4,Overall material and finish of the house,7,12776.3,Total square feet of basement area,963,-5103.03
268,178734.474531,Ground living area square feet,2192,29747,Overall material and finish of the house,5,-26151.3,Overall condition of the house,8,9190.84
289,113950.844570,Overall material and finish of the house,5,-24730,Ground living area square feet,900,-16342.6,Total square feet of basement area,882,-5922.64
650,74957.162142,Overall material and finish of the house,4,-33927.7,Ground living area square feet,630,-23234.4,Total square feet of basement area,630,-11687.9
1234,135305.243500,Overall material and finish of the house,5,-25445.7,Ground living area square feet,1188,-11476.6,Condition of sale,Abnormal Sale,-5071.82


#### Save SmartExplainer in Pickle File

You can save the SmartExplainer Object in a pickle file to make new plots later or launch the WebApp again

In [17]:
xpl.save('./xpl.pkl')